# Tekton

In [2]:
!minikube start

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Automatically selected the docker driver. Other choices: none, ssh
📌  Using Docker driver with root privileges
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring bridge CNI (Container Networking Interface) ...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🔎  Verifying Kubernetes components...
🌟  Enabled addons: default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


In [3]:
%%bash

kubectl apply --filename https://storage.googleapis.com/tekton-releases/pipeline/latest/release.yaml

# install tkn cli
curl -LO https://github.com/tektoncd/cli/releases/download/v0.32.0/tektoncd-cli-0.32.0_Linux-64bit.deb
sudo dpkg -i ./tektoncd-cli-0.32.0_Linux-64bit.deb
rm ./tektoncd-cli-0.32.0_Linux-64bit.deb

# grant all cluster role permissions to default service account
kubectl apply -f ./k8s/default-cluster-role.yml
kubectl apply -f ./k8s/default-cluster-role-binding.yml

namespace/tekton-pipelines created
clusterrole.rbac.authorization.k8s.io/tekton-pipelines-controller-cluster-access created
clusterrole.rbac.authorization.k8s.io/tekton-pipelines-controller-tenant-access created
clusterrole.rbac.authorization.k8s.io/tekton-pipelines-webhook-cluster-access created
clusterrole.rbac.authorization.k8s.io/tekton-events-controller-cluster-access created
role.rbac.authorization.k8s.io/tekton-pipelines-controller created
role.rbac.authorization.k8s.io/tekton-pipelines-webhook created
role.rbac.authorization.k8s.io/tekton-pipelines-events-controller created
role.rbac.authorization.k8s.io/tekton-pipelines-leader-election created
role.rbac.authorization.k8s.io/tekton-pipelines-info created
serviceaccount/tekton-pipelines-controller created
serviceaccount/tekton-pipelines-webhook created
serviceaccount/tekton-events-controller created
clusterrolebinding.rbac.authorization.k8s.io/tekton-pipelines-controller-cluster-access created
clusterrolebinding.rbac.authorizati

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 28.5M  100 28.5M    0     0  36.0M      0 --:--:-- --:--:-- --:--:-- 36.0M


(Reading database ... 70203 files and directories currently installed.)
Preparing to unpack .../tektoncd-cli-0.32.0_Linux-64bit.deb ...
Unpacking cli (0.32.0) over (0.32.0) ...
Setting up cli (0.32.0) ...
clusterrole.rbac.authorization.k8s.io/full-access-clusterrole created
clusterrolebinding.rbac.authorization.k8s.io/tekton-kubectl-binding created


In [4]:
!kubectl get pods --namespace tekton-pipelines

NAME                                           READY   STATUS    RESTARTS      AGE
tekton-events-controller-5994bb6b74-kc8pr      1/1     Running   1 (67s ago)   2d18h
tekton-pipelines-controller-6b984488dc-75sf6   1/1     Running   1 (67s ago)   2d18h
tekton-pipelines-webhook-d55cc5fb7-5z5lj       1/1     Running   1 (67s ago)   2d18h


### Task 1: Build

In [139]:
%%writefile tasks/build.yml
apiVersion: tekton.dev/v1beta1
kind: Task
metadata:
  name: build
spec:
  params:
    - name: REPO_URL
      type: string
  steps:
    - name: clone-repo
      image: node
      script: |
        git clone $(params.REPO_URL) /workspace/source
    - name: npm-install
      image: node
      workingDir: /workspace/source
      script: |
        npm install
    - name: npm-test
      image: node
      workingDir: /workspace/source
      script: |
        npm test


Overwriting tasks/build.yml


In [140]:
!kubectl apply -f tasks/build.yml

task.tekton.dev/build configured


In [141]:
!tkn task list

NAME     DESCRIPTION   AGE
build                  1 hour ago
deploy                 29 minutes ago
push                   11 minutes ago


In [142]:
!tkn task describe build

Name:        build
Namespace:   default

⚓ Params

 NAME         TYPE     DESCRIPTION   DEFAULT VALUE
 ∙ REPO_URL   string                 ---

🦶 Steps

 ∙ clone-repo
 ∙ npm-install
 ∙ npm-test


In [143]:
%%bash
# cd ./tekton
kubectl apply -f ./tasks/build.yml

# reason why its 5 and not 3 is output includes `Steps: ` and new line
NUM_STEPS=`tkn task describe build | grep Steps -A 10 | wc -l`
if [ "$NUM_STEPS" == "5" ]; then
echo "✅ build is 3 steps"
else
echo "❌ build is not 3 steps"
fi

CLONE=`kubectl describe task build | grep 'git clone' -o`
if [ "$CLONE" == "git clone" ]; then
echo "✅ git clone step included"
else
echo "❌ git clone step not included"
fi

INSTALL=`kubectl describe task build | grep 'npm install' -o`
if [ "$INSTALL" == "npm install" ]; then
echo "✅ npm install step included"
else
echo "❌ npm install step not included"
fi

TEST=`kubectl describe task build | grep 'npm test' -o`
if [ "$TEST" == "npm test" ]; then
echo "✅ npm test step included"
else
echo "❌ npm test step not included"
fi

task.tekton.dev/build configured
✅ build is 3 steps
✅ git clone step included
✅ npm install step included
✅ npm test step included


### Task 2: Push

In [144]:
!kubectl apply -f ./tasks/push.yml

task.tekton.dev/push configured


### Task 3: Deploy

For now, the `tasks/deploy.yml` file assumes that we have pushed up `nginx:latest` image that we then update for the `k8s/nginx-deployment.yml`.  Please complete the `tasks/deploy.yml` file by adding the following Steps to your Task:

   a. Clone the repo specified by `REPO_URL` in `/workspace/source/`

   b. Run  `kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml`

   c. Run: `kubectl set image deployment/nginx-deployment nginx=<IMAGE-NAME> -n=<NAMESPACE>` where `IMAGE_NAME` and `NAMESPACE` are passed as parameters to our Task.

   Once you are ready to test run `kubectl apply -f ./tasks/deploy.yml` and you can see its creation status by running `tkn task list`.

In [145]:
%%writefile tasks/deploy.yml
apiVersion: tekton.dev/v1beta1
kind: Task
metadata:
  name: deploy
spec:
  params:
    - name: NAMESPACE
      type: string
    - name: IMAGE
      type: string
    - name: REPO_URL
      type: string
  steps:
    - name: clone-repo
      image: node
      script: |
        git clone $(params.REPO_URL) /workspace/source
    - name: apply-deployment
      image: lachlanevenson/k8s-kubectl
      script: |
        kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml
    - name: update-image
      image: lachlanevenson/k8s-kubectl
      script: |
        kubectl set image deployment/nginx-deployment nginx=$(params.IMAGE) -n=$(params.NAMESPACE)

Overwriting tasks/deploy.yml


In [146]:
%%bash 

kubectl apply -f ./tasks/deploy.yml

NUM_STEPS=`tkn task describe deploy | grep Steps -A 10 | wc -l`
if [ "$NUM_STEPS" == "5" ]; then
echo "✅ deploy is 3 steps"
else
echo "❌ deploy is not 3 steps"
fi

CLONE=`kubectl describe task deploy | grep 'git clone' -o`
if [ "$CLONE" == "git clone" ]; then
echo "✅ git clone step included"
else
echo "❌ git clone step not included"
fi

APPLY=`kubectl describe task deploy | grep 'kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml' -o`
if [ "$APPLY" == "kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml" ]; then
echo "✅ kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml step included"
else
echo "❌ kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml step not included"
fi

SET_IMAGE=`kubectl describe task deploy | grep 'kubectl set image' -o`
if [ "$SET_IMAGE" == "kubectl set image" ]; then
echo "✅ kubectl set image step included"
else
echo "❌ kubectl set image step not included"
fi

task.tekton.dev/deploy configured
✅ deploy is 3 steps
✅ git clone step included
✅ kubectl apply -f /workspace/source/tekton/k8s/nginx-deployment.yml step included
✅ kubectl set image step included


### Task 4: Pipeline

In [147]:
%%writefile build-push-deploy-pipeline.yml
apiVersion: tekton.dev/v1beta1
kind: Pipeline
metadata:
  name: build-push-deploy
spec:
  tasks:
  - name: build
    taskRef:
      name: build
    params: 
      - name: REPO_URL
        value: https://github.com/expressjs/express.git
  - name: push
    taskRef:
      name: push
    params:
    - name: IMAGE_NAME
      value: nginx:latest
    - name: DOCKERFILE_PATH
      value: /workspace/source
    - name: CONTEXT_PATH
      value: /workspace/source
    - name: CONTAINER_REGISTRY    
      value: registry.example.com
    - name: REGISTRY_USERNAME
      value: Bob
    - name: REGISTRY_PASSWORD 
      value: password1234
  - name: deploy
    taskRef:
      name: deploy
    params:
    - name: NAMESPACE
      value: default
    - name: IMAGE 
      value: nginx:latest
    - name: REPO_URL
      value: https://github.com/abanuelo/intro-to-kube.git

Overwriting build-push-deploy-pipeline.yml


In [148]:
%%bash
kubectl apply -f ./build-push-deploy-pipeline.yml
NUM_STEPS=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | wc -l`
if [ "$NUM_STEPS" == "6" ]; then
echo "✅ pipeline has 3 tasks"
else
echo "❌ pipeline does not have 3 tasks"
fi

NUM_STEPS=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | wc -l`
if [ "$NUM_STEPS" == "6" ]; then
echo "✅ pipeline has 3 tasks"
else
echo "❌ pipeline does not have 3 tasks"
fi

FIRST=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==4 {print; exit}' | grep 'build' -o | tr -d '\n'`
if [ "$FIRST" == "buildbuild" ]; then
echo "✅ build is first task in pipeline"
else
echo "❌ build is not first task in pipeline"
fi

SECOND=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' | grep 'push' -o | tr -d '\n'`
if [ "$SECOND" == "pushpush" ]; then
echo "✅ push is second task in pipeline"
else
echo "❌ push is not second task in pipeline"
fi

THIRD=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==6 {print; exit}' | grep 'deploy' -o | tr -d '\n'`
if [ "$THIRD" == "deploydeploy" ]; then
echo "✅ deploy is third task in pipeline"
else
echo "❌ deploy is not third task in pipeline"
fi

# ENV VARIABLES FOR BUILD
BUILD_ENV=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==4 {print; exit}' | grep 'REPO_URL: https://github.com/expressjs/express.git' -o`
if [ "$BUILD_ENV" == "REPO_URL: https://github.com/expressjs/express.git" ]; then
echo "✅ build task has correct REPO_URL"
else
echo "❌ build task does not have correct REPO_URL"
fi

# ENV VARIABLES FOR PUSH
PUSH_ENV_1=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' |  grep 'IMAGE_NAME: nginx:latest' -o`
if [ "$PUSH_ENV_1" == "IMAGE_NAME: nginx:latest" ]; then
echo "✅ push task has correct IMAGE_NAME"
else
echo "❌ push task does not have correct IMAGE_NAME"
fi
PUSH_ENV_2=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' |  grep 'DOCKERFILE_PATH: /workspace/source' -o`
if [ "$PUSH_ENV_2" == "DOCKERFILE_PATH: /workspace/source" ]; then
echo "✅ push task has correct DOCKERFILE_PATH"
else
echo "❌ push task does not have correct DOCKERFILE_PATH"
fi
PUSH_ENV_3=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' |  grep 'CONTEXT_PATH: /workspace/source' -o`
if [ "$PUSH_ENV_3" == "CONTEXT_PATH: /workspace/source" ]; then
echo "✅ push task has correct CONTEXT_PATH"
else
echo "❌ push task does not have correct CONTEXT_PATH"
fi
PUSH_ENV_4=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' |  grep 'CONTAINER_REGISTRY: registry.example.com' -o`
if [ "$PUSH_ENV_4" == "CONTAINER_REGISTRY: registry.example.com" ]; then
echo "✅ push task has correct CONTAINER_REGISTRY"
else
echo "❌ push task does not have correct CONTAINER_REGISTRY"
fi
PUSH_ENV_5=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' |  grep 'REGISTRY_USERNAME: Bob' -o`
if [ "$PUSH_ENV_5" == "REGISTRY_USERNAME: Bob" ]; then
echo "✅ push task has correct REGISTRY_USERNAME"
else
echo "❌ push task does not have correct REGISTRY_USERNAME"
fi
PUSH_ENV_6=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==5 {print; exit}' |  grep 'REGISTRY_PASSWORD: password1234' -o`
if [ "$PUSH_ENV_6" == "REGISTRY_PASSWORD: password1234" ]; then
echo "✅ push task has correct REGISTRY_PASSWORD"
else
echo "❌ push task does not have correct REGISTRY_PASSWORD"
fi

# ENV VARIABLES FOR DEPLOY
DEPLOY_ENV_1=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==6 {print; exit}' | grep 'NAMESPACE: default' -o`
if [ "$DEPLOY_ENV_1" == "NAMESPACE: default" ]; then
echo "✅ deploy task has correct NAMESPACE"
else
echo "❌ deploy task does not have correct NAMESPACE"
fi
DEPLOY_ENV_2=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==6 {print; exit}' | grep 'IMAGE: nginx:latest' -o`
if [ "$DEPLOY_ENV_2" == "IMAGE: nginx:latest" ]; then
echo "✅ deploy task has correct IMAGE"
else
echo "❌ deploy task does not have correct IMAGE"
fi
DEPLOY_ENV_3=`tkn pipeline describe build-push-deploy | grep 'Tasks' -A 10 | awk 'NR==6 {print; exit}' | grep 'REPO_URL: https://github.com/abanuelo/intro-to-kube.git' -o`
if [ "$DEPLOY_ENV_3" == "REPO_URL: https://github.com/abanuelo/intro-to-kube.git" ]; then
echo "✅ deploy task has correct REPO_URL"
else
echo "❌ deploy task does not have correct REPO_URL"
fi

pipeline.tekton.dev/build-push-deploy created
✅ pipeline has 3 tasks
✅ pipeline has 3 tasks
✅ build is first task in pipeline
✅ push is second task in pipeline
✅ deploy is third task in pipeline
✅ build task has correct REPO_URL
✅ push task has correct IMAGE_NAME
✅ push task has correct DOCKERFILE_PATH
✅ push task has correct CONTEXT_PATH
✅ push task has correct CONTAINER_REGISTRY
✅ push task has correct REGISTRY_USERNAME
✅ push task has correct REGISTRY_PASSWORD
✅ deploy task has correct NAMESPACE
✅ deploy task has correct IMAGE
✅ deploy task has correct REPO_URL


In [150]:
%%bash
kubectl apply -f tasks/build.yml
kubectl apply -f tasks/push.yml
kubectl apply -f tasks/deploy.yml
kubectl apply -f build-push-deploy-pipeline.yml
kubectl apply -f build-push-deploy-pipelinerun.yml

task.tekton.dev/build configured
task.tekton.dev/push configured
task.tekton.dev/deploy configured
pipeline.tekton.dev/build-push-deploy configured
pipelinerun.tekton.dev/build-push-deploy created


In [151]:
!tkn pipelinerun logs build-push-deploy -f -n default

[push : push] NOTE: FOR CHALLENGE EXERCISE ONLY
[push : push] ===========================================
[push : push] This task does not push anything to the Docker's Container Registry
[push : push] If you wish to do so, add the extension to minikube: https://minikube.sigs.k8s.io/docs/handbook/registry/
[push : push] And modify the password, username, and registry links in the build-push-deploy.yml Pipeline
[push : push] If you wish the automate the push of a real, existing repo, also change the REPO_URL in build.yml and ensure a Dockerfile is present at its root
[push : push] /tekton/scripts/script-0-kv5ll: line 9: nginx-deployment.yml: not found
[push : push] Afterwards edit the  file to point to your image instead of the nginx default
[push : push] ===========================================
[push : push] Credentials
[push : push] ===========================================
[push : push] 
[push : push] 
[push : push] 

[deploy : clone-repo] Cloning into '/workspace/source'...
[bu

In [152]:
%%bash

kubectl apply -f ./build-push-deploy-pipelinerun.yml

sleep 100

tkn pipelinerun describe build-push-deploy

SUCCESS_FULL=`tkn pipelineruns describe build-push-deploy | grep 'Status' -A 4 | grep 'Succeeded' -o`
if [ "$SUCCESS_FULL" == "Succeeded" ]; then
echo "✅ build-push-deploy succeeded"
else
echo "❌ build-push-deploy errored"
fi

SUCCESS_BUILD=`tkn pipelineruns describe build-push-deploy | grep 'build-push-deploy-build' | grep 'Succeeded' -o`
if [ "$SUCCESS_BUILD" == "Succeeded" ]; then
echo "✅ build-push-deploy build task succeeded"
else
echo "❌ build-push-deploy build task errored"
fi

SUCCESS_PUSH=`tkn pipelineruns describe build-push-deploy | grep 'build-push-deploy-push' | grep 'Succeeded' -o`
if [ "$SUCCESS_PUSH" == "Succeeded" ]; then
echo "✅ build-push-deploy push task succeeded"
else
echo "❌ build-push-deploy push task errored"
fi

SUCCESS_DEPLOY=`tkn pipelineruns describe build-push-deploy | grep 'build-push-deploy-deploy' | grep 'Succeeded' -o`
if [ "$SUCCESS_DEPLOY" == "Succeeded" ]; then
echo "✅ build-push-deploy deloy task succeeded"
else
echo "❌ build-push-deploy deploy task errored"
fi

pipelinerun.tekton.dev/build-push-deploy unchanged
Name:              build-push-deploy
Namespace:         default
Pipeline Ref:      build-push-deploy
Service Account:   default
Labels:
 tekton.dev/pipeline=build-push-deploy

Status

STARTED         DURATION   STATUS
3 minutes ago   1m8s       Succeeded

Timeouts
 Pipeline:   1h0m0s

Taskruns

 NAME                       TASK NAME   STARTED         DURATION   STATUS
 build-push-deploy-build    build       3 minutes ago   1m7s       Succeeded
 build-push-deploy-push     push        3 minutes ago   13s        Succeeded
 build-push-deploy-deploy   deploy      3 minutes ago   25s        Succeeded
✅ build-push-deploy succeeded
✅ build-push-deploy build task succeeded
✅ build-push-deploy push task succeeded
✅ build-push-deploy deloy task succeeded


In [1]:
!minikube delete

🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /home/codespace/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.
